<h1> <center> RETRHO PSF Photometry Tool </center> </h1>
<h2> <center> Timeseries Analysis </center> </h2>

This notebook provides an example of how to visualize and analyze the photometric measurements using the RETRHO PSF Photometry Tool for one or muttiple nights. This notebook will show you how you can:

1. Visualize the PSF contour around the selected stars
2. Visualize the measured flux for the main target as well as the different reference stars to facilitate the reference star selection
3. Compute and display the flux ratio between the main target and the reference stars with the propagated uncertainties

**NOTE**
This tool is more helpful if it's used to measure and visualize the photometric measurements of:

- Exoplanet Transits
- Variable Stars
- Supernovae (could vary depending on the ovearll brightness. Further tests need to be done)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.visualization import ZScaleInterval, ImageNormalize
from astropy.stats import sigma_clipped_stats
from scipy import ndimage
import os


## Import the data

We start by loading up the directory (or directories) where the reduced files of a given object are. If the same object was observed during multiple nights, then add a directory to the list. 

If you observed the object for mutiple nights, make sure to select the same reference stars in the same order for the different observing nights

In [ ]:
# Define the directory where the reduced files and the PSF photometry results are stored
# the format of this list is: ['path/to/data_directory1/', 'path/to/data_directory2/', ...]
data_directory_list = [] # Change based on the date and object name and add more directories if needed

In [ ]:
# Interpolate over multiple directories to extract the data and create a composite dataframe
dataframes = []

for data_directory in data_directory_list:
    # Define the PSF photometry results directory and filename
    psf_photometry_directory = os.path.join(data_directory, 'PSF_Photometry_Results/')
    psf_photometry_filename = [f for f in os.listdir(psf_photometry_directory) if f.endswith('_psf_photometry.csv')][0]

    # Open the PSF photometry Dataframe
    photometry_data = pd.read_csv(os.path.join(psf_photometry_directory, psf_photometry_filename))

    # Add the data directory as a new column at the beginning to identify the night
    photometry_data.insert(0, 'Data_Directory', data_directory)
    dataframes.append(photometry_data)

# Concatenate all dataframes into a single dataframe
psf_photometry_df = pd.concat(dataframes, ignore_index=True)

print(f"There are a total of {len(psf_photometry_df)} photometric measurements from {len(data_directory_list)} observing nights.")
psf_photometry_df.head()

## Visualize Star Positions and the PSF Contours

Once you have the composte dataframe with all the photometric measurements, you can display the different stars that were used to measure photometry for and display the computed PSF. You can change the `image_index` variable to select a different image to display. 

The first cell bellow will show the full reduced frame along with the different stars selected for photometric calculations and their respective IDs. The cell after that shows a sub-plot zoomed in into each individual star to trace the PSF contour that was used to measure the star flux

In [ ]:
image_index = 5  # Change this index to read different images

In [ ]:
# Read in the image files based on index number. Remember that index = 0 represents the first image
image_file = os.path.join(psf_photometry_df.loc[image_index, 'Data_Directory'], psf_photometry_df.loc[image_index, 'File'])

# Figure out how many stars were analyzed in the PSF photometry
num_stars = len([col for col in psf_photometry_df.columns if col.startswith('Flux_Star_')])

# Get the X, Y positions of the stars from the DataFrame
star_positions = [(psf_photometry_df.loc[image_index, f'Star_{i+1}_x'], \
                   psf_photometry_df.loc[image_index, f'Star_{i+1}_y']) \
                   for i in range(num_stars)
                ]

# Extract basic information from the FITS header
header = fits.getheader(image_file)
object_name = header.get('OBJECT', 'Unknown')
filter_ = header.get('FILTER', 'Unknown')
exptime = header.get('EXPTIME', 1.0)

print(f"Object Name: {object_name}")
print(f"Exposure Time: {exptime} seconds")
print(f"Filter: {filter_}")
print(f"Total selected stars for PSF photometry: {num_stars}")

# Extract the image data and normalize it to ZScale
image_data = fits.getdata(image_file)
norm = ImageNormalize(image_data, interval=ZScaleInterval())

# Plot the image with star positions marked
fig, ax = plt.subplots(1,1, figsize=(10, 10))
ax.imshow(image_data, origin='lower', cmap='gray', norm=norm)

for i, (x, y) in enumerate(star_positions):
    ax.plot(x, y, 'rx', markersize=15, markeredgewidth=1)
    ax.text(x + 10, y + 10, f'Star {i+1}', color='yellow', fontsize=12)

ax.set_title(f"Star Positions PSF Photometry - {os.path.basename(image_file)}", size=14)

ax.axis('off')
plt.show()

In [ ]:
# Define the star window size
width, height = 40, 40

# Define the figure to plot the PSF
if num_stars % 2 == 0:
    ncols = 2
    nrows = num_stars // 2
else:
    ncols = 3
    nrows = (num_stars + 2) // 3
fig, ax = plt.subplots(nrows, ncols, figsize=(10, 10), tight_layout=True)

# Interpolate over the star peaks to isolate the PSF
for i, (xc, yc) in enumerate(star_positions):
    
    # Create the star cutout for each star
    star_cutout = image_data[yc - height//2:yc + height//2, xc - width//2:xc + width//2]
    
    # Calculate the statistics of the star cutout and define the threshold
    mean, median, std = sigma_clipped_stats(star_cutout, sigma=3.0)
    threshold = median + 3 * std

    # Create initial mask of pixels above threshold
    bright_pixels = star_cutout > threshold
    
    # Label connected regions to identify separate bright areas
    labeled_regions, num_regions = ndimage.label(bright_pixels)
    
    # Define the center coordinates in the cutout
    center_y, center_x = height//2, width//2
    
    # Find which region contains the center
    center_region = labeled_regions[center_y, center_x]
    
    # If center is not in a bright region, find the nearest one
    if center_region == 0:
        y, x = np.ogrid[:star_cutout.shape[0], :star_cutout.shape[1]]
        distance = np.sqrt((x - center_x)**2 + (y - center_y)**2)
        bright_positions = np.where(bright_pixels)

        if len(bright_positions[0]) > 0:
            # Find the closest bright pixel to the center
            dist_to_bright = distance[bright_positions]
            closest_idx = np.argmin(dist_to_bright)
            center_region = labeled_regions[bright_positions[0][closest_idx], 
                                            bright_positions[1][closest_idx]]
    
    # Create mask for the central region
    central_region = labeled_regions == center_region
    
    # Combine masks to get final PSF selection
    final_mask = central_region

    # Plot the PSF for each star
    ax[i].imshow(star_cutout, norm=norm, origin='lower', cmap='gray')
    ax[i].contour(final_mask, levels=[0.5], colors='red', linewidths=2)

    ax[i].set_title(f'Star {i+1} PSF')
    ax[i].axis('off')

plt.show()


## Visualize the Measured Stellar Flux and Flux Ratio
We then show how to display the measured flux of the all the stars to facilitate the reference star selection. Keep in mind that Star 1 should always be your main target.

The cell bellow shows the measured flux from the inside of the PSF contour with the respective uncertainties for each star

In [ ]:
# Define the figure to plot the measured flux of each star over time
fig, ax = plt.subplots(figsize=(10, 6))

# Interpolate over the number of stars to plot their fluxes
for i in range(num_stars):
    ax.errorbar(psf_photometry_df['BJD'], psf_photometry_df[f'Flux_Star_{i+1}'], \
                yerr=psf_photometry_df[f'Flux_err_Star_{i+1}'], fmt='.', \
                capsize=3, label=f'Star {i+1}')

ax.set_xlabel('BJD')
ax.set_ylabel('Flux')
ax.legend()

plt.show()

Finally, we look at the flux ratio between the main target relative to a reference star. You can change the variables `main_star_id` and `ref_star_id` to select the ID for the main target and the reference target respectively.

The flux ratio is then propagated from the flux uncertainty from each star as:

$\sigma_{F_{ratio}} = F_{ratio} \sqrt{(\frac{\sigma_{F_{main}}}{F_{main}})^2 + (\frac{\sigma_{F_{ref}}}{F_{ref}})^2}$

In [ ]:
# Define the main and reference star identifiers to compute the flux ratio
main_star_id = 1  # Main target star
ref_star_id = 2 # Change this based on which star you want as reference


In [ ]:
# Measure the flux ratio between the main target and the reference star and the associated error
flux_ratio = psf_photometry_df[f'Flux_Star_{main_star_id}'] / psf_photometry_df[f'Flux_Star_{ref_star_id}']
flux_ratio_err = flux_ratio * np.sqrt((psf_photometry_df[f'Flux_err_Star_{main_star_id}'] / psf_photometry_df[f'Flux_Star_{main_star_id}'])**2. + \
                                      (psf_photometry_df[f'Flux_err_Star_{ref_star_id}'] / psf_photometry_df[f'Flux_Star_{ref_star_id}'])**2.)


# Plot the flux ratio over time
fig, ax = plt.subplots(figsize=(8, 5), tight_layout=True)
ax.set_title(f'Flux Ratio {object_name}')
ax.errorbar(psf_photometry_df['BJD'], flux_ratio, yerr=flux_ratio_err, fmt='.', label='PSF Photometry', color='k', capsize=3)
ax.set_xlabel('BJD')
ax.set_ylabel(f'Flux Ratio (Star {main_star_id} / Star {ref_star_id})')
plt.show()
